In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
import torch
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import math
from sklearn.model_selection import train_test_split,cross_val_score,KFold,StratifiedKFold
import torchtuples as tt

In [2]:
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis
from sksurv.preprocessing import OneHotEncoder, encode_categorical

from sklearn import set_config
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
datas = pd.read_csv('veteran.csv')
datas

,trt,celltype,time,status,karno,diagtime,age,prior
0,1,squamous,72,1,60,7,69,0
1,1,squamous,411,1,70,5,64,10
2,1,squamous,228,1,60,3,38,0
3,1,squamous,126,1,60,9,63,10
4,1,squamous,118,1,70,11,65,10
...,...,...,...,...,...,...,...,...
132,2,large,133,1,75,1,65,0
133,2,large,111,1,60,5,64,0
134,2,large,231,1,70,18,67,10
135,2,large,378,1,80,4,65,0


In [4]:
data_y = datas.loc[:, ['time', 'status']].copy()
data_y = data_y.rename(columns={'status': 'cens'})
data_x = datas.drop(['time', 'status'], axis=1)

data_x['celltype'] = data_x['celltype'].astype('category')
data_x = OneHotEncoder().fit_transform(data_x)
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3, random_state=1)
list_of_tuples = list(y_train.itertuples(index=False, name=None))
swapped_list = [(t[1], t[0]) for t in list_of_tuples]
y_train = np.array(swapped_list, dtype=[('cens', bool), ('time', float)])
list_of_tuples = list(y_test.itertuples(index=False, name=None))
swapped_list = [(t[1], t[0]) for t in list_of_tuples]
y_test = np.array(swapped_list, dtype=[('cens', bool), ('time', float)])

In [5]:
for df in [X_train, X_test]:
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].min()) / (df[numeric_columns].max() - df[numeric_columns].min())

In [6]:
X_test

,trt,celltype=large,celltype=smallcell,celltype=squamous,karno,diagtime,age,prior
80,1.0,0.0,0.0,1.0,0.000000,1.000000,0.805556,0.0
5,0.0,0.0,0.0,1.0,0.000000,0.114286,0.416667,0.0
39,0.0,0.0,1.0,0.0,0.506329,0.200000,0.777778,0.0
36,0.0,0.0,1.0,0.0,0.126582,0.085714,0.722222,0.0
35,0.0,0.0,1.0,0.0,0.506329,0.685714,0.888889,1.0
58,0.0,1.0,0.0,0.0,0.506329,0.314286,0.805556,0.0
44,0.0,0.0,1.0,0.0,0.253165,0.628571,0.916667,1.0
77,1.0,0.0,0.0,1.0,0.506329,0.057143,0.666667,0.0
120,1.0,0.0,0.0,0.0,0.886076,0.057143,0.722222,0.0
119,1.0,0.0,0.0,0.0,0.632911,0.057143,0.805556,0.0


# MODELS

In [7]:
cox_lasso = CoxnetSurvivalAnalysis(l1_ratio=1, alpha_min_ratio=0.01)
cox_lasso.fit(X_train, y_train)
cox_lasso.score(X_test, y_test)

0.7128834355828221

In [8]:
cox_ridge = CoxnetSurvivalAnalysis(l1_ratio=0.00001, alpha_min_ratio=0.01)
cox_ridge.fit(X_train, y_train)
cox_ridge.score(X_test, y_test)

0.7061349693251534

In [9]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

sns.set_style("white")

import torch # For building the networks
import torchtuples as tt # Some useful functions
from torch import nn
import torch.nn.functional as F

from pycox.datasets import support
from pycox.models import DeepHitSingle
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv
from pycox.preprocessing.feature_transforms import OrderedCategoricalLong
from pycox.models.loss import rank_loss_deephit_single

In [10]:
data_y_transform_train = pd.DataFrame.from_records(y_train, columns=['event', 'time'])
data_y_transform_test = pd.DataFrame.from_records(y_test, columns=['event', 'time'])
data_y_transform_train['cens'] = data_y_transform_train['cens'].replace({True: 1, False: 0})
data_y_transform_test['cens'] = data_y_transform_test['cens'].replace({True: 1, False: 0})
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train_values= X_train.values
X_test_values= X_test.values

In [11]:
num_durations = 10

labtrans = DeepHitSingle.label_transform(num_durations)
get_target = lambda df: (df['time'].values, df['cens'].values)
y_train_deephit = labtrans.fit_transform(*get_target(data_y_transform_train))
durations_test, events_test = get_target(data_y_transform_test)

In [12]:
in_features = X_train.shape[1]
out_features = labtrans.out_features
num_nodes = [32]
batch_norm = True
dropout = 0.2

net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout)
optimizer = tt.optim.AdamWR(decoupled_weight_decay=0.01, cycle_eta_multiplier=0.8,
                            cycle_multiplier=2)
model_deep_hit = DeepHitSingle(net, optimizer, alpha = 0.2, sigma = 0.1, duration_index=labtrans.cuts)
epochs = 30
batch_size=8
model_deep_hit.fit(X_train_values, y_train_deephit, batch_size, epochs)
surv = model_deep_hit.predict_surv_df(X_test_values)
ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
ev.concordance_td('antolini')

C:\Users\User\anaconda3\lib\site-packages\torchtuples\callbacks.py:607: UserWarning: This overload of add is deprecated:
	add(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add(Tensor other, *, Number alpha) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:1025.)
  p.data = p.data.add(-weight_decay * eta, p.data)


0:	[0s / 0s],		train_loss: 0.7640
1:	[0s / 0s],		train_loss: 0.7025
2:	[0s / 0s],		train_loss: 0.6625
3:	[0s / 0s],		train_loss: 0.6710
4:	[0s / 0s],		train_loss: 0.6179
5:	[0s / 0s],		train_loss: 0.6562
6:	[0s / 0s],		train_loss: 0.6677
7:	[0s / 0s],		train_loss: 0.6430
8:	[0s / 0s],		train_loss: 0.6222
9:	[0s / 0s],		train_loss: 0.6200
10:	[0s / 0s],		train_loss: 0.6041
11:	[0s / 0s],		train_loss: 0.6047
12:	[0s / 1s],		train_loss: 0.5625
13:	[0s / 1s],		train_loss: 0.5732
14:	[0s / 1s],		train_loss: 0.5887
15:	[0s / 1s],		train_loss: 0.5371
16:	[0s / 1s],		train_loss: 0.5730
17:	[0s / 1s],		train_loss: 0.5851
18:	[0s / 1s],		train_loss: 0.5479
19:	[0s / 1s],		train_loss: 0.5734
20:	[0s / 1s],		train_loss: 0.5830
21:	[0s / 1s],		train_loss: 0.5196
22:	[0s / 1s],		train_loss: 0.5427
23:	[0s / 1s],		train_loss: 0.5416
24:	[0s / 1s],		train_loss: 0.5403
25:	[0s / 1s],		train_loss: 0.5332
26:	[0s / 1s],		train_loss: 0.5265
27:	[0s / 1s],		train_loss: 0.5201
28:	[0s / 1s],		train_loss: 0.

0.505521472392638

In [13]:
from sksurv.ensemble import RandomSurvivalForest
rsf = RandomSurvivalForest(n_estimators=1000,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           n_jobs=-1,
                           random_state=42)
rsf.fit(X_train,y_train)
rsf.score(X_test, y_test)

0.6766871165644172

In [14]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
estimator = CoxPHSurvivalAnalysis().fit(X_train, y_train)
estimator.score(X_test,y_test)

0.7067484662576687

In [15]:
from sksurv.linear_model import CoxnetSurvivalAnalysis
estimator = CoxnetSurvivalAnalysis(l1_ratio=0.99, fit_baseline_model=True)
estimator.fit(X_train, y_train)
estimator.score(X_test, y_test)

0.7104294478527607

In [16]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
est_cph_tree = GradientBoostingSurvivalAnalysis(
    n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0
)
est_cph_tree.fit(X_train, y_train)
cindex = est_cph_tree.score(X_test, y_test)
cindex

0.49447852760736194

In [37]:
import torch
import torchtuples as tt

from pycox.datasets import metabric
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

In [45]:
get_target = lambda df: (df['time'].values, df['cens'].values)
y_train_deepsurv = get_target(data_y_transform_train)
durations_test, events_test = get_target(data_y_transform_test)

In [46]:
in_features = X_train.shape[1]
num_nodes = [64, 64]
out_features = 1
batch_norm = True
dropout = 0.2
output_bias = False

net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                              dropout, output_bias=output_bias)

In [47]:
optimizer = tt.optim.AdamWR(decoupled_weight_decay=0.01, cycle_eta_multiplier=0.8,
                            cycle_multiplier=2)
model_deepsurv = CoxPH(net, optimizer)
batch_size= 8
epochs= 100
verbose=1
callbacks = [tt.callbacks.EarlyStopping()]
log = model_deepsurv.fit(X_train_values, y_train_deepsurv, batch_size, epochs, callbacks, verbose)
model_deepsurv.compute_baseline_hazards()
surv = model_deepsurv.predict_surv_df(X_test_values)
ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
ev.concordance_td()

0:	[0s / 0s],		train_loss: 1.3669
1:	[0s / 0s],		train_loss: 1.2659
2:	[0s / 0s],		train_loss: 1.2547
3:	[0s / 0s],		train_loss: 1.2545
4:	[0s / 0s],		train_loss: 1.3367
5:	[0s / 0s],		train_loss: 1.1347
6:	[0s / 0s],		train_loss: 1.2188
7:	[0s / 0s],		train_loss: 1.2109
8:	[0s / 0s],		train_loss: 1.1784
9:	[0s / 0s],		train_loss: 1.2351


0.4805352798053528

In [41]:
from pycox.models import LogisticHazard
# from pycox.models import PMF
# from pycox.models import DeepHitSingle
from pycox.evaluation import EvalSurv

In [42]:
num_durations = 10

labtrans = LogisticHazard.label_transform(num_durations)
# labtrans = PMF.label_transform(num_durations)
# labtrans = DeepHitSingle.label_transform(num_durations)

get_target = lambda df: (df['time'].values, df['cens'].values)
y_train_loghazard = labtrans.fit_transform(*get_target(data_y_transform_train))
y_test_loghazard = labtrans.transform(*get_target(data_y_transform_test))


# # We don't need to transform the test labels
durations_test, events_test = get_target(data_y_transform_test)

In [43]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train_values= X_train.values
X_test_values= X_test.values

In [44]:
in_features = X_train.shape[1]
num_nodes = [32, 32]
out_features = labtrans.out_features
batch_norm = True
dropout = 0.1
net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout)
model = LogisticHazard(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts)
batch_size = 32
epochs = 20
log= model.fit(X_train_values, y_train_loghazard, batch_size, epochs)
surv = model.predict_surv_df(X_test_values)
ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
ev.concordance_td('antolini')

0:	[0s / 0s],		train_loss: 1.7519
1:	[0s / 0s],		train_loss: 1.5183
2:	[0s / 0s],		train_loss: 1.3705
3:	[0s / 0s],		train_loss: 1.2594
4:	[0s / 0s],		train_loss: 1.1596
5:	[0s / 0s],		train_loss: 1.0294
6:	[0s / 0s],		train_loss: 0.9398
7:	[0s / 0s],		train_loss: 0.8500
8:	[0s / 0s],		train_loss: 0.8237
9:	[0s / 0s],		train_loss: 0.7187
10:	[0s / 0s],		train_loss: 0.7363
11:	[0s / 0s],		train_loss: 0.6214
12:	[0s / 0s],		train_loss: 0.6094
13:	[0s / 0s],		train_loss: 0.5629
14:	[0s / 0s],		train_loss: 0.5221
15:	[0s / 0s],		train_loss: 0.5787
16:	[0s / 0s],		train_loss: 0.4882
17:	[0s / 0s],		train_loss: 0.4683
18:	[0s / 0s],		train_loss: 0.4711
19:	[0s / 0s],		train_loss: 0.4935


0.5067484662576687

In [48]:
cox= CoxnetSurvivalAnalysis()
rsf = RandomSurvivalForest(n_estimators=1000,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           random_state=42)
cox.fit(X_train_values, y_train)
rsf.fit(X_train_values, y_train)
cox_pred = cox.predict(X_test_values)
rsf_pred = rsf.predict(X_test_values)
ensemble_pred= (cox_pred+rsf_pred)/2
from sksurv.metrics import concordance_index_censored
cindex = concordance_index_censored(y_test["cens"], y_test["time"], ensemble_pred)
print("Concordance Index:", cindex[0])

Concordance Index: 0.6785276073619632


In [49]:
cox= CoxPHSurvivalAnalysis()
rsf = RandomSurvivalForest(n_estimators=1000,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           random_state=42)
cox_lasso = CoxnetSurvivalAnalysis(l1_ratio=1, alpha_min_ratio=0.01)
cox_lasso.fit(X_train, y_train)
cox.fit(X_train_values, y_train)
rsf.fit(X_train_values, y_train)
lasso_predict = cox_lasso.predict(X_test_values)
cox_pred = cox.predict(X_test_values)
rsf_pred = rsf.predict(X_test_values)
ensemble_pred = np.maximum(cox_pred, rsf_pred, lasso_predict)
from sksurv.metrics import concordance_index_censored
cindex = concordance_index_censored(y_test["cens"], y_test["time"], ensemble_pred)
print("Concordance Index:", cindex[0])

Concordance Index: 0.6766871165644172


C:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but CoxnetSurvivalAnalysis was fitted with feature names
  warnings.warn(
